In [1]:
import scipy.signal
import numpy
import sys
sys.path.append('..')

In [2]:
import lasp.differential
import lasp.utils
import lasp.filters.linear

In [3]:
arr = numpy.reshape(numpy.arange(0, 16), (4, 4))
arr_fft = numpy.fft.fft2(arr) # img in frequency domain
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


## dx (with periodic boundary ie circular convolution)

In [4]:
# Derivation
#dx_oper = numpy.array([[ 1, -1 ]]) # operator
kernel2d_id = numpy.pad(numpy.array([[1]]), pad_width=1)
dx_oper = lasp.differential.dx(kernel2d_id)


## With convolution (ie spatial domain)
arr_dx_v1 = scipy.signal.convolve2d(
    arr, 
    dx_oper, 
    mode='same',
    boundary='circular'
)


## With point wise product (THIS IS NOT CONVOLITION CIRCULARY)
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dx_oper_pad = lasp.utils.pad(dx_oper, arr.shape)
print('padded :\n', dx_oper_pad)
dx_oper_pad_fft = numpy.fft.fft2(dx_oper_pad) # operator in frequency domain
arr_dx_fft_v2 = dx_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dx_v2 = numpy.real(numpy.fft.ifft2(arr_dx_fft_v2)) # Return in sptial


# ## BCCB tricks
dx_oper_diag = lasp.utils.fourier_diagonalization(dx_oper, arr.shape)
arr_fft_dx_v3 = dx_oper_diag * arr_fft
arr_dx_v3 = numpy.real(numpy.fft.ifft2(arr_fft_dx_v3))


## With numpy
arr_dx_v4 = lasp.differential.dx(arr)

print('Kernel identity :\n', kernel2d_id)
print('dx_oper :\n', dx_oper)
axis = 1
n= kernel2d_id.shape[axis]
#print('numpy.diff :\n', numpy.transpose(kernel2d_id[:, 0]))
#print('numpy.diff :\n', numpy.diff(kernel2d_id, prepend=numpy.transpose(kernel2d_id[:, 0])))
#print('conv2d :\n', arr_dx_v1)
#print('spatial :\n', arr_dx_v2)
#print('BCCB :\n', arr_dx_v3)
#print('Direct :\n', arr_dx_v4)
#numpy.reshape(kernel2d_id[:, n-1], (-1, 1))


padded :
 [[ 0.  0.  0.  0.]
 [ 0.  1. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
Kernel identity :
 [[0 0 0]
 [0 1 0]
 [0 0 0]]
dx_oper :
 [[ 0.  0.  0.]
 [ 0.  1. -1.]
 [ 0.  0.  0.]]


## dy

In [5]:
# Derivation
dy_oper = numpy.transpose(numpy.array([[ 1, -1 ]])) # operator


## With convolution (ie spatial domain)
arr_dy_v1 = scipy.signal.convolve2d(
    arr, 
    dy_oper, 
    mode ='same',
    boundary = 'wrap'
)


## With point wise product
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dy_oper_pad = lasp.utils.pad(dy_oper, arr.shape)
dy_oper_pad_fft = numpy.fft.fft2(dy_oper_pad) # operator in frequency domain
arr_dy_fft_v2 = dy_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dy_v2 = numpy.real(numpy.fft.ifft2(arr_dy_fft_v2)) # Return in sptial


# ## BCCB tricks
dy_oper_diag = lasp.utils.fourier_diagonalization(dy_oper, arr.shape)
arr_fft_dy_v3 = dy_oper_diag * arr_fft
arr_dy_v3 = numpy.real(numpy.fft.ifft2(arr_fft_dy_v3))


## With numpy
arr_dy_v4 = lasp.differential.dy(arr)

print(arr_dy_v1)
print(arr_dy_v2)
print(arr_dy_v3)
print(arr_dy_v4)

[[-12 -12 -12 -12]
 [  4   4   4   4]
 [  4   4   4   4]
 [  4   4   4   4]]
[[-12. -12. -12. -12.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]]
[[  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [-12. -12. -12. -12.]]
[[-12. -12. -12. -12.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]]


## dxT

In [6]:
# Derivation

dx_oper = numpy.array([[ 1, -1 ]]) # operator

## With point wise product
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dx_oper_pad = lasp.utils.pad(dx_oper, arr.shape)
dx_oper_pad_fft = numpy.fft.fft2(dx_oper_pad)
dxT_oper_pad_fft = numpy.conj(dx_oper_diag)
arr_dxT_fft_v1 = dxT_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dxT_v1 = numpy.real(numpy.fft.ifft2(arr_dxT_fft_v1)) # Return in sptial


# ## BCCB tricks
# Derivation
dx_oper_diag = lasp.utils.fourier_diagonalization(dx_oper, arr.shape)
dxT_oper_diag = numpy.conj(dx_oper_diag)
arr_fft_dxT_v2 = dxT_oper_diag * arr_fft
arr_dxT_v2 = numpy.real(numpy.fft.ifft2(arr_fft_dxT_v2))


## With numpy
arr_dxT_v3 = lasp.differential.dxT(arr)



print('Convolution :', 'Not spatial operator')
print('Hadamard in Fourier:\n', arr_dxT_v1)
print('BCCB :\n', arr_dxT_v2)
print('dxT :\n', arr_dxT_v3)



Convolution : Not spatial operator
Hadamard in Fourier:
 [[-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]]
BCCB :
 [[ 3. -1. -1. -1.]
 [ 3. -1. -1. -1.]
 [ 3. -1. -1. -1.]
 [ 3. -1. -1. -1.]]
dxT :
 [[-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]]


## dyT

In [7]:
# Derivation

dy_oper = numpy.transpose(numpy.array([[ 1, -1 ]])) # operator

## With point wise product
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dy_oper_pad = lasp.utils.pad(dy_oper, arr.shape)
dy_oper_pad_fft = numpy.fft.fft2(dy_oper_pad)
dyT_oper_pad_fft = numpy.conj(dy_oper_diag)
arr_dyT_fft_v1 = dyT_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dyT_v1 = numpy.real(numpy.fft.ifft2(arr_dyT_fft_v1)) # Return in sptial


# ## BCCB tricks
# Derivation
dy_oper_diag = lasp.utils.fourier_diagonalization(dy_oper, arr.shape)
dyT_oper_diag = numpy.conj(dy_oper_diag)
arr_fft_dyT_v2 = dyT_oper_diag * arr_fft
arr_dyT_v2 = numpy.real(numpy.fft.ifft2(arr_fft_dyT_v2))


## With numpy
arr_dyT_v3 = lasp.differential.dyT(arr)

print('Convolution :', 'Not spatial operator')
print('Hadamard in Fourier:\n', arr_dyT_v1)
print('BCCB :\n', arr_dyT_v2)
print('dxT :\n', arr_dyT_v3)

Convolution : Not spatial operator
Hadamard in Fourier:
 [[12. 12. 12. 12.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]]
BCCB :
 [[12. 12. 12. 12.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]]
dxT :
 [[-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [12. 12. 12. 12.]]


## Laplacian

In [8]:
x_oper = numpy.array([[ 1, -1 ]]) # operator
dx_oper_pad = lasp.utils.pad(dx_oper, arr.shape)
dx_oper_pad_fft = numpy.fft.fft2(dx_oper_pad)
dxT_oper_pad_fft = numpy.conj(dx_oper_diag)

dy_oper = numpy.transpose(numpy.array([[ 1, -1 ]])) # operator
dy_oper_pad = lasp.utils.pad(dy_oper, arr.shape)
dy_oper_pad_fft = numpy.fft.fft2(dy_oper_pad)
dyT_oper_pad_fft = numpy.conj(dy_oper_diag)

lap_pad_fft_v1 = \
    dxT_oper_pad_fft * dx_oper_pad_fft \
    + dyT_oper_pad_fft * dy_oper_pad_fft

arr_lap_fft_v1 = lap_pad_fft_v1 * arr_fft
arr_lap_v1 = numpy.real(numpy.fft.ifft(arr_lap_fft_v1))

###########################

lap_filter = lasp.filters.linear.laplacian()
print('lap filter :\n', lap_filter)

lap_pad_v2 = lasp.utils.pad(lap_filter, arr.shape)
lap_pad_fft_v2 = numpy.fft.fft2(lap_pad_v2)

arr_lap_fft_v2 = lap_pad_fft_v2 * arr_fft
arr_lap_v2 = numpy.real(numpy.fft.ifft2(arr_lap_fft_v2))

###########################

print('\nResults :\n')
print('Lap v1 :\n', arr_lap_v1)
print('lap v2 :\n', arr_lap_v2)

lap filter :
 [[ 0 -1  0]
 [-1  4 -1]
 [ 0 -1  0]]

Results :

Lap v1 :
 [[ 16. -16.   0.   0.]
 [ 16.  16.  16.  16.]
 [ 32.  32.  32.  32.]
 [ 16.  16.  16.  16.]]
lap v2 :
 [[ 20.  12.  16.  16.]
 [-12. -20. -16. -16.]
 [  4.  -4.   0.   0.]
 [  4.  -4.   0.   0.]]


In [9]:
laplacian = lasp.filters.linear.laplacian()
lap_diag_v1 = lasp.utils.fourier_diagonalization(
    kernel = laplacian,
    shape_out = arr.shape 
)

arr_lap_fft_v1 = lap_diag_v1 * arr_fft
arr_lap_v1 = numpy.real(numpy.fft.ifft(arr_lap_fft_v1))

Dx_pad = lasp.utils.pad(numpy.array([[1, -1]]), arr.shape)
Dx = numpy.fft.fft2(Dx_pad)
Dxt = numpy.conj(Dx)

Dy_pad = lasp.utils.pad(numpy.transpose(numpy.array([[1, -1]])), arr.shape)
Dy = numpy.fft.fft2(Dy_pad)
Dyt = numpy.conj(Dy)

Dx_diag = lasp.utils.fourier_diagonalization(
    kernel = Dx_pad,
    shape_out = arr.shape 
)
Dxt_diag = numpy.conj(Dx_diag)

Dy_diag = lasp.utils.fourier_diagonalization(
    kernel = Dy_pad,
    shape_out = arr.shape 
)
Dyt_diag = numpy.conj(Dy_diag)

lap_diag_v2 = Dxt_diag * Dx_diag + Dyt_diag * Dy_diag

arr_lap_fft_v1 = lap_diag_v2 * arr_fft
arr_lap_v1 = numpy.real(numpy.fft.ifft(arr_lap_fft_v2))

###

F2D = Dxt*Dx+Dyt*Dy

print('With laplacian filter :\n', arr_lap_v1)
print('With laplacian definition :\n', arr_lap_v2)
print((F2D == lap_diag_v2))

With laplacian filter :
 [[ 16. -16.   0.   0.]
 [ 16.  16.  16.  16.]
 [ 32.  32.  32.  32.]
 [ 16.  16.  16.  16.]]
With laplacian definition :
 [[ 20.  12.  16.  16.]
 [-12. -20. -16. -16.]
 [  4.  -4.   0.   0.]
 [  4.  -4.   0.   0.]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [10]:
## Test
kernel2d_id = numpy.pad(numpy.array([[1]]), pad_width=1)
dx_oper = lasp.differential.dx(kernel2d_id)
arr_id = scipy.signal.convolve2d(arr, kernel2d_id, mode='same')

arr_dx = scipy.signal.convolve2d(arr, dx_oper, mode='same', boundary='wrap')
print(arr_dx)

[[-3.  1.  1.  1.]
 [-3.  1.  1.  1.]
 [-3.  1.  1.  1.]
 [-3.  1.  1.  1.]]


In [40]:
kernel3d_id = numpy.pad(numpy.array([[[1]]]), pad_width=1)

n = kernel3d_id.shape[0]


tmp = numpy.stack((, kernel3d_id[n-1, :, :]), axis=0)

numpy.diff(tmp, prepend= axis=0)


ValueError: all input arrays must have the same shape

In [11]:
def derivation(arr: numpy.ndarray, axis: int) -> numpy.ndarray:
    shape = numpy.copy(arr.shape)
    n = shape[axis]
    taken = numpy.take(arr, axis=axis, indices=n-1)
    shape[axis] = 1
    to_add = numpy.reshape(taken, shape)
    return numpy.diff(arr, prepend=to_add, axis=axis)

In [13]:
kernel2d_id = numpy.pad(numpy.array([[1]]), pad_width=1)
kernel2d_id

array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [15]:
kernel3d_id = numpy.pad(numpy.array([[[1]]]), pad_width=1)
kernel3d_id

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]])

In [16]:
print(derivation(kernel2d_id, 1))
print(lasp.differential.dx(kernel2d_id))

[[ 0  0  0]
 [ 0  1 -1]
 [ 0  0  0]]
[[ 0.  0.  0.]
 [ 0.  1. -1.]
 [ 0.  0.  0.]]


In [17]:
print(derivation(kernel2d_id, 0))
print(lasp.differential.dy(kernel2d_id))

[[ 0  0  0]
 [ 0  1  0]
 [ 0 -1  0]]
[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]]


In [18]:
print(derivation(kernel3d_id, 1))

[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  1  0]
  [ 0 -1  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]


In [19]:
print(derivation(kernel3d_id, 0))

[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  1  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0 -1  0]
  [ 0  0  0]]]


In [20]:
print(derivation(kernel3d_id, 2))

[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  1 -1]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]


In [86]:
d = 3
zeros = numpy.zeros((d, d, d))
center = lasp.utils.compute_center(zeros)
zeros[ tuple(center)] = 1
zeros

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [88]:
numpy.full(shape=(1, d), fill_value=1)

array([[3, 3, 3]])

In [95]:
d = 2
shape = numpy.full(shape=d, fill_value=1)
print(shape)
tshape = tuple(shape)
print(tshape)
one_nd = numpy.full(shape=tuple(shape), fill_value=1)
numpy.pad(array=one_nd, pad_width=1)

[1 1]
(1, 1)


array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [100]:
def kernel_identity(dim: int) -> numpy.array:
    shape = numpy.full(shape=dim, fill_value=1)
    one_nd = numpy.full(shape=tuple(shape), fill_value=1)
    return numpy.pad(array=one_nd, pad_width=1)

In [107]:
ker_id_2d = kernel_identity(dim=2)
ker_id_2d

array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [108]:
d_0 = derivation(ker_id_2d, axis=0)
d_0

array([[ 0,  0,  0],
       [ 0,  1,  0],
       [ 0, -1,  0]])

In [109]:
d_1 = derivation(ker_id_2d, axis=1)
d_1

array([[ 0,  0,  0],
       [ 0,  1, -1],
       [ 0,  0,  0]])